In [2]:
#Loading the data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import warnings
warnings.filterwarnings('ignore')



directory = 'dataset/'

# First concat the complete data

dataset = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(os.path.join(directory, filename))
        # Concatenate the DataFrame to the existing one
        dataset = pd.concat([dataset, df], ignore_index=True)


In [6]:
# dataset["Participant"][1000]

50

In [3]:
dataset_grouped = dataset.groupby("Participant")

In [8]:
for participant, group in dataset_grouped:
    print("Participant:", participant)
    print(group)
    print("\n")


Participant: 7
         Unnamed: 0  RecordingTime [ms] Time of Day [h:m:s:ms]     Trial  \
1126368       69722         5491055.316           10:10:48:020  Trial002   
1126369       69723         5491095.284           10:10:48:060  Trial002   
1126370       69724         5491139.400           10:10:48:104  Trial002   
1126371       69725         5491183.403           10:10:48:148  Trial002   
1126372       69726         5491227.403           10:10:48:192  Trial002   
...             ...                 ...                    ...       ...   
2064174       73435         5490743.043           10:10:47:707  Trial001   
2064175       73436         5490787.041           10:10:47:751  Trial001   
2064176       73437         5490831.159           10:10:47:795  Trial001   
2064177       73438         5490875.157           10:10:47:839  Trial001   
2064178       73439         5490919.163           10:10:47:883  Trial001   

                                     Stimulus  Export Start Trial Time [

In [10]:
num_groups = dataset_grouped.ngroups

print("Number of groups:", num_groups)


Number of groups: 97


In [20]:
groups_list = dataset_grouped.groups.keys()

print("List of groups:")
groups = []
for group in groups_list:
    if isinstance(group, int): groups.append(group)
    print(group)


List of groups:
7
11
14
15
17
18
19
20
21
22
23
24
25
26
27
28
29
30
35
37
38
39
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
58
59
1
10
11
13
14
15
17
18
19
2
20
21
22
23
24
25
26
27
28
29
3
30
31
32
33
34
35
36
37
38
39
4
40
42
43
44
45
46
47
48
49
5
50
51
52
53
54
55
56
57
58
59
6
7
8
9
Unidentified(Neg)
Unidentified(Pos)


In [22]:
groups.sort()

In [23]:
groups

[7,
 11,
 14,
 15,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 35,
 37,
 38,
 39,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 58,
 59]

In [24]:
len(groups)

39

## Grouping by sessions

In [46]:
prevID = dataset["Participant"][0]

# sessions = pd.DataFrame()
sessions = []
currSession = pd.DataFrame()
# for row in dataset:
#     if row[] != prevID:
#         sessions.insert()
#     else:
#         currSession.append(row)

for i in range(len(dataset)):
    if dataset["Participant"][i] != prevID:
        print("Session #", len(sessions))
        print(len(currSession))

        # sessions = pd.concat([sessions, currSession], ignore_index=True)
        sessions.append(currSession)

    prevID = dataset["Participant"][i]
    currSession = pd.concat([currSession, pd.DataFrame([dataset.iloc[i]])], ignore_index=True)


Session # 0
2625
Session # 1
5229
Session # 2
7685
Session # 3
10314
Session # 4
12918
Session # 5
15501
Session # 6
17205
Session # 7
19841
Session # 8
22203
Session # 9
24801
Session # 10
27323
Session # 11
29946
Session # 12
31285
Session # 13
32768
Session # 14
32770
Session # 15
35668
Session # 16
38063
Session # 17
40543
Session # 18
42564
Session # 19
44718
Session # 20
45963
Session # 21
47594
Session # 22
49152
Session # 23
49653
Session # 24
51359
Session # 25
52842
Session # 26
55081
Session # 27
58345
Session # 28
60478
Session # 29
62648
Session # 30
65560
Session # 31
68494
Session # 32
71378
Session # 33
74300
Session # 34
77238
Session # 35
80173
Session # 36
83095
Session # 37
86038
Session # 38
88981
Session # 39
91883
Session # 40
94801
Session # 41
97745
Session # 42
97774
Session # 43
100715
Session # 44
121291
Session # 45
124231
Session # 46
125992
Session # 47
127739
Session # 48
129543
Session # 49
131582
Session # 50
133510
Session # 51
135496
Session # 52
137